In [1]:
from rich.pretty import pprint

## The CMA-ES Benchmark
Let's take a look at CMA-ES. This is a benchmark for both step-size control and algorithm variation selection of CMA-ES. First, let's make in instance of the benchmark:

In [2]:
from dacbench.benchmarks import CMAESBenchmark
bench = CMAESBenchmark()

Now let's take a look at the elements of the config in this benchmark:

In [3]:
pprint(list(bench.config.keys()))

[
│   'config_space',
│   'observation_space_class',
│   'observation_space_args',
│   'observation_space_type',
│   'reward_range',
│   'budget',
│   'cutoff',
│   'seed',
│   'multi_agent',
│   'instance_set_path',
│   'test_set_path',
│   'benchmark_info'
]

The 'benchmark_info' tells us some things about this benchmark already:

In [4]:
pprint(bench.config["benchmark_info"])

{
│   'identifier': 'ModCMA',
│   'name': 'Online Selection of CMA-ES Variants',
│   'reward': 'Negative best function value',
│   'state_description': ['Generation Size', 'Sigma', 'Remaining Budget', 'Function ID', 'Instance ID']
}

The 'config_space' specifies which actions are taken, i.e. which hyperparameters are configured. We can see that this is a large and complex configuration space with mainly categorical values and then the step size float.

In [5]:
pprint(bench.config["config_space"])

Configuration space object:
  Hyperparameters:
│   0_active, Type: Categorical, Choices: {True, False}, Default: True
│   1_elitist, Type: Categorical, Choices: {True, False}, Default: True
│   2_orthogonal, Type: Categorical, Choices: {True, False}, Default: True
│   3_sequential, Type: Categorical, Choices: {True, False}, Default: True
│   4_threshold_convergence, Type: Categorical, Choices: {True, False}, Default: True
│   5_step_size_adaptation, Type: Categorical, Choices: {csa, tpa, msr, xnes, m-xnes, lp-xnes, psr}, Default: csa
│   6_mirrored, Type: Categorical, Choices: {None, mirrored, mirrored pairwise}, Default: None
│   7_base_sampler, Type: Categorical, Choices: {gaussian, sobol, halton}, Default: gaussian
│   8_weights_option, Type: Categorical, Choices: {default, equal, 1/2^lambda}, Default: default
│   90_local_restart, Type: Categorical, Choices: {None, IPOP, BIPOP}, Default: None
│   91_bound_correction, Type: Categorical, Choices: {None, saturate, unif_resample, COTN, toroidal, mirror}, Default: None
│   92_step_size, Type: UniformFloat, Range: [0.0, 10.0], Default: 5.0

The reward in this task has the following reward range:

In [6]:
pprint(bench.config["reward_range"])

(-1000000000000, 0)

Finally, the cutoff shows how many steps of the sequence are necessary for solution:

In [7]:
pprint(bench.config["cutoff"])

1000000.0

The config also contains some standard keys like the seed, instance set, multi agent mode or observation space config. The observation space usually does not need to be configured at all while the seed should be varied between runs. Multi agent mode has its own example which better shows how it works.

## CMA-ES Instances
Now let's take a look at how a CMA-ES instance looks. To do so, we first read the default instance set and look at its first element:

In [12]:
pprint(bench.config["instance_set_path"])
bench.read_instance_set()
pprint(bench.config.instance_set[0])
pprint(f"Target function: {bench.config.instance_set[0].target_function.function}")

'../instance_sets/cma/cma_bbob_dim10_train.csv'

CMAESInstance(
│   target_function=<dacbench.envs.env_utils.toy_functions.IOHFunction object at 0x1740b27d0>,
│   dim=10,
│   fid=3,
│   iid=1,
│   active=False,
│   elitist=False,
│   orthogonal=True,
│   sequential=True,
│   threshold_convergence=False,
│   step_size_adaptation='psr',
│   mirrored=nan,
│   base_sampler='sobol',
│   weights_option='1/2^lambda',
│   local_restart='IPOP',
│   bound_correction=nan
)

'Target function: <RealSingleObjectiveProblem 14. DifferentPowers (iid=1 dim=10)>'

As you can see, the instance contains the "target_function", in this case the DifferentPowers function with id 1 in dimension 10. Natively this benchmark supports BBOB functions via the "read_instance_set" method, but it could be overwritten by an IOHFunction. Furthermore it also contains default values for the algorithm component choices, e.g. "active" or "elitist". They serve as defaults in case only a subset of the configuration space is configured.

## Running CMA-ES
Lastly, let's look at the CMA-ES benchmark in action. The state contains the size of the current generation, the current step size, the remaining budget, function ID and instance ID:

In [13]:
env = bench.get_environment()
pprint(env.reset())

(array([ 10.,   2., 100.,  10.,   4.]), {})

If we take a step, we see the updated step size and budget:

In [14]:
action = env.action_space.sample()
state, reward, terminated, truncated, info = env.step(action)
pprint(state)

array([10.        ,  9.34079924, 94.        , 10.        ,  4.        ])

Furthermore, we also get a reward and termination and truncation signals. Truncation will be set to true after the number of steps exceeds the cutoff.

In [15]:
pprint(f"Reward {reward}")
pprint(f"Terminated {terminated}")
pprint(f"Truncated {truncated}")

'Reward 0'

'Terminated False'

'Truncated False'